## This script will allow you to connect your local machine to BigQuery on Google Cloud
### Please note that you will have to setup a service account (as in the tutorial) within your Google Cloud account and download a private key (JSON file). You will also need your Project-ID and ensure you have all the access rights.
### The JSON file will be used for authentication with BigQuery. 

In [18]:
pip install --upgrade google-cloud-bigquery

Requirement already up-to-date: google-cloud-bigquery in c:\users\harsh\anaconda3\lib\site-packages (1.24.0)
Note: you may need to restart the kernel to use updated packages.


## Change the credentials as follows:
### Change the key path to your private key
### Change the project-id to your project-id

In [3]:
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file(
    'C:/Users/harsh/Desktop/MSBA/Spring/Big Data/Project/key.json.json')
project_id = 'managing-big-data-project'
client = bigquery.Client(credentials= credentials,project=project_id)

### You can modify the SQL query in any way you prefer. Ensure the table format is project-id.database.table

In [4]:
sql = """
  SELECT * FROM `managing-big-data-project.Political_Ads.advertiser_weekly_spend` """

In [5]:
df = client.query(sql).to_dataframe()

In [6]:
df.head()

,advertiser_id,advertiser_name,election_cycle,week_start_date,spend_usd,spend_eur,spend_inr,spend_bgn,spend_hrk,spend_czk,spend_dkk,spend_huf,spend_pln,spend_ron,spend_sek,spend_gbp
0,AR100207290732249088,FRIENDS OF SUZY GLOWIAK,US-Federal-2018,2018-09-16,0,0,0,0,0,0,0,0,0,0,0,0
1,AR100207290732249088,FRIENDS OF SUZY GLOWIAK,US-Federal-2018,2018-09-23,0,0,250,0,0,0,0,0,0,0,0,0
2,AR100207290732249088,FRIENDS OF SUZY GLOWIAK,US-Federal-2018,2018-09-30,0,0,500,0,0,0,0,0,0,0,0,0
3,AR101183107301900288,Fianna Fáil,None,2020-01-05,0,0,0,0,0,0,0,0,0,0,0,0
4,AR101487259705933824,MANUEL E LLOSAS,US-Federal-2018,2019-09-29,0,0,0,0,0,0,0,0,0,0,0,0


## More complex querying

### Retrieve only the data we need from BigQuery, instead of loading all tables into machine memory for manipulation

In [25]:
sql = """SELECT * FROM(
SELECT advertiser_name,elections, CONCAT("$",CAST(Total_Spending AS INT64)) as Spending, 
ROW_NUMBER() OVER (PARTITION BY elections ORDER BY Total_Spending DESC) AS rank
FROM(
SELECT advertiser_name, elections, sum(Total_Spending_USD) as Total_Spending
FROM(
SELECT advertiser_name,elections, regions, (spend_usd + spend_eur/0.893 + spend_inr/70.394 + spend_bgn/1.66 + spend_hrk/6.72 + spend_czk/21.49 + spend_dkk/6.61 + spend_huf/285.81 + spend_pln/3.78 + spend_ron/4.18 + spend_sek/9.41 + spend_gbp/0.784) as Total_Spending_USD
FROM `managing-big-data-project.Political_Ads.advertiser_stats`
WHERE elections IS NOT NULL
)
GROUP BY advertiser_name,elections
ORDER BY sum(Total_Spending_USD) DESC))
WHERE rank <= 5
ORDER BY elections DESC"""

In [29]:
result = client.query(sql).to_dataframe() #Sends the query to BigQuery and saves result in a dataframe

In [28]:
result

,advertiser_name,elections,Spending,rank
0,MIKE BLOOMBERG 2020 INC,US-Federal,$459665851,1
1,TRUMP MAKE AMERICA GREAT AGAIN COMMITTEE,US-Federal,$152565798,2
2,TOM STEYER 2020,US-Federal,$87737257,3
3,"PETE FOR AMERICA, INC.",US-Federal,$77930512,4
4,"DONALD J. TRUMP FOR PRESIDENT, INC.",US-Federal,$65597912,5
5,Bharatiya Janata Party,IN-General,$24027440,1
6,DRAVIDA MUNNETRA KAZHAGAM,IN-General,$6935651,2
7,Auburn Digital Solutions Private Limited,IN-General,$5901308,3
8,Indian National Congress,IN-General,$5216979,4
9,Yuvajana Sramika Rythu Congress Party,IN-General,$2774190,5
